In [1]:
import pandas as pd
import optuna
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from catboost import CatBoostRegressor
from tqdm import tqdm
import warnings
from datetime import datetime, timedelta

In [2]:
original_df = pd.read_csv('to_predict', sep='\t')
original_df['lcl_dt'] = pd.to_datetime(original_df['lcl_dt'])

# Функция для добавления временных признаков
def add_time_features(df):
    df['day_of_week'] = df['lcl_dt'].dt.dayofweek
    df['week_of_year'] = df['lcl_dt'].dt.isocalendar().week
    df['month_of_year'] = df['lcl_dt'].dt.month
    df['week_of_month'] = df['lcl_dt'].apply(lambda x: (x.day - 1) // 7 + 1)
    return df.copy()

# Объединенная функция для создания лагов и расчета суммы продаж
def create_lagged_sales_sums(df, days):
    df = df.sort_values(by=['city_name', 'item_id', 'lcl_dt'])
    
    # Создание лагов
    for day in range(1, days + 1):
        df[f'sales_lag_{day}'] = df.groupby(['city_name', 'item_id'])['sales'].shift(day)
        df[f'osa_perc_lag_{day}'] = df.groupby(['city_name', 'item_id'])['osa_perc'].shift(day)
        df[f'avg_price_lag_{day}'] = df.groupby(['city_name', 'item_id'])['avg_price'].shift(day)
        df[f'future_price_lag_{day}'] = df.groupby(['city_name', 'item_id'])['avg_price'].shift(-day)
    
    # Расчет целевых переменных
    df['future_sales_sum'] = df.groupby(['city_name', 'item_id'])['sales'].transform(lambda x: x.rolling(window=days, min_periods=1).sum().shift(-days+1))
    df['past_sales_sum'] = df.groupby(['city_name', 'item_id'])['sales'].transform(lambda x: x.rolling(window=days).sum())
    
    return df.copy()

# Определение количества дней для лагов
days = 7

# Добавляем временные признаки
df = add_time_features(original_df)

# Создаем лаги и целевые переменные
df = create_lagged_sales_sums(df, days)

df.dropna(subset=[f'future_price_lag_1', f'future_sales_sum', 'past_sales_sum'])

df[['lvl3_category_name', 'lvl4_subcategory_name', 'lvl5_subcategory_name', 'city_name']] =\
df[['lvl3_category_name', 'lvl4_subcategory_name', 'lvl5_subcategory_name', 'city_name']].fillna('None')

In [3]:
# Предполагая, что lcl_dt уже в формате datetime в формате date
yesterday = pd.to_datetime(datetime.today().date() - timedelta(days=1))

# Фильтрация записей на вчерашний день
filtered_df = df[df['lcl_dt'] == yesterday]

In [5]:
filtered_df.shape

(2852, 44)

In [6]:
filtered_df['retail_price_with_nds'].head()

41529    319.0
41530    185.0
41531    159.0
41538    159.0
41539    115.0
Name: retail_price_with_nds, dtype: float64

In [7]:
# Пример сетки цен
price_grid = [
  1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 17, 19, 22, 
  25, 27, 29, 31, 33, 35, 37, 39, 42, 45, 47, 49, 53, 55,
  59, 65, 69, 75, 79, 85, 89, 95, 99, 105, 109, 115, 119, 
  125, 129, 135, 139, 145, 149, 155, 159, 165, 169, 175, 
  179, 185, 189, 199, 205, 209, 215, 219, 225, 229, 235, 
  239, 245, 249, 255, 259, 265, 269, 275, 279, 285, 289, 
  299, 309, 319, 329, 339, 349, 359, 369, 379, 389, 399, 
  409, 419, 429, 439, 449, 459, 469, 479, 489, 499, 509, 
  519, 529, 539, 549, 559, 569, 579, 589, 599, 609, 619, 
  629, 639, 649, 659, 669, 679, 689, 699, 709, 719, 729, 
  739, 749, 759, 769, 779, 789, 799, 809, 819, 829, 839, 
  849, 859, 869, 879, 889, 899, 909, 919, 929, 939, 949, 
  959, 969, 979, 989, 999, 1019, 1039, 1059, 1079, 1099, 
  1119, 1139, 1159, 1179, 1199, 1219, 1239, 1259, 1279, 
  1299, 1319, 1339, 1359, 1379, 1399, 1419, 1439, 1459, 
  1479, 1499, 1519, 1539, 1559, 1579, 1599, 1619, 1639, 
  1659, 1679, 1699, 1719, 1739, 1759, 1779, 1799, 1819, 
  1839, 1859, 1879, 1899, 1919, 1939, 1959, 1979, 1999, 
  2019, 2039, 2059, 2079, 2099, 2119, 2139, 2159, 2179, 
  2199
  ]

# Создание нового DataFrame
result_rows = []

for _, row in filtered_df.iterrows():
    item_id = row['item_id']
    current_price = row['retail_price_with_nds']
    
    # Вычисление нижней и верхней границы цены
    lower_bound = current_price * 0.9
    upper_bound = current_price * 1.1
    
    # Поиск цен в пределах 10%
    valid_prices = [price for price in price_grid if lower_bound <= price <= upper_bound]
    
    for price in valid_prices:
        new_row = row.to_dict()  # Копируем оригинальную строку как словарь
        new_row['avg_price'] = price  # Добавляем новое значение цены
        result_rows.append(new_row)

# Создание результирующего DataFrame
result_df = pd.DataFrame(result_rows)

In [9]:
lag_features = [f'sales_lag_{day}' for day in range(1, days + 1)] \
             + [f'osa_perc_lag_{day}' for day in range(1, days + 1)] \
             + [f'avg_price_lag_{day}' for day in range(1, days + 1)]

categorical_features = ['lvl3_category_name', 'lvl4_subcategory_name', 'lvl5_subcategory_name', 'city_name']
time_features = ['day_of_week', 'week_of_year', 'month_of_year', 'week_of_month']
numerical_features = ['avg_price', 'past_sales_sum']

features = numerical_features + lag_features + categorical_features + time_features

X = result_df[features]

In [10]:
model = CatBoostRegressor()
model.load_model("catboost_model.cbm")

In [11]:
result_df['predictions'] = model.predict(X)

# Применяем метод clip к столбцу 'predictions' с верхним лимитом 2
result_df['predictions'] = result_df['predictions'].clip(upper=1.5)

result_df['new_sales_abs'] = result_df['predictions'] * result_df['past_sales_sum']
result_df['new_gmv'] = result_df['new_sales_abs'] * result_df['avg_price']

In [13]:
max_gmv_per_item = result_df.groupby('item_id')['new_gmv'].transform('max')
max_gmv_per_item.head()

0    55734.934672
1    55734.934672
2    55734.934672
3    55734.934672
4    55734.934672
Name: new_gmv, dtype: float64

In [14]:
# Оставляем только те строки, у которых new_gmv равно максимальному значению для данного item_id
result_df = result_df[result_df['new_gmv'] == max_gmv_per_item]
result_df['new_sales'] = result_df['predictions']-1
result_df['optimizer_price'] = result_df['avg_price']

In [20]:
columns_to_save = ['item_id', 'new_sales', 'optimizer_price']

In [24]:
result_df[columns_to_save].reset_index(drop=True).to_excel('optimizer_results.xlsx')